<a href="https://colab.research.google.com/github/zayo1511/LaboratorioDeAnalisisDeDatosFinancierosYDeDisenoDeIndicadores/blob/main/clase10_prediccion_ltv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Algoritmo para predecir el LTV a 24 meses

### Preparación de los datos

1. **Colección y limpieza de datos:**
    - Asegurarse de que los datos incluyan información relevante como demografía del cliente, historial de compras y métricas de interacción.
    - Limpiar los datos para manejar valores faltantes, outliers y consistencia de formatos.
    - Define y calcula la variable objetivo.

### Ingeniería de características

2. **Creación de características:**
    - Crear características que reflejen el comportamiento del cliente, tales como el gasto total, la cantidad promedio de transacciones, la frecuencia de compra, y el tiempo desde la última compra.
    - Considerar la creación de características en ventanas de tiempo como el gasto total en los últimos 6 o 12 meses.

### Análisis exploratorio de datos (EDA)

3. **Análisis exploratorio:**
    - Analizar los datos para entender patrones y relaciones.
    - Visualizar las distribuciones del gasto y otras métricas relevantes a través de diferentes segmentos de clientes.
    - Agrupar clientes basándose en la fecha de su primera compra y seguir su comportamiento a lo largo del tiempo.


### Modelado predictivo

4. **Modelado predictivo:**
    - Dividir los datos en conjuntos de entrenamiento y prueba.
    - Elegir un modelo de regresión adecuado para predecir un resultado continuo como el LTV.
    - Entrenar el modelo en los datos de entrenamiento utilizando las características desarrolladas.

### Evaluación del modelo

5. **Evaluación del modelo:**
    - Evaluar el modelo en el conjunto de prueba usando métricas adecuadas, como el error cuadrático medio (RMSE) o el error absoluto medio (MAE).
    - Analizar los residuales para asegurar que el modelo funciona bien en todos los segmentos de clientes.

### Interpretación del modelo y aplicación de insights

6. **Interpretación y acción:**
    - Interpretar el modelo para entender qué características son más predictivas del LTV.
    - Usar la salida del modelo para segmentar clientes basándose en su LTV predicho y dirigirles estrategias de marketing específicas.

### Despliegue y monitoreo

7. **Despliegue y monitoreo:**
    - Implementar el modelo en un entorno de producción donde pueda predecir el LTV de nuevos clientes.
    - Monitorear regularmente el desempeño del modelo y actualizarlo según sea necesario para manejar cambios en el comportamiento del cliente o en las condiciones del mercado.

In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm
from google.colab import drive
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/Colab Notebooks/Laboratorio de analisis de datos financieros y de diseño de indicadores/Analisis estadistico avanzado para KPIs/'

Mounted at /content/drive


In [2]:
df_data = pd.read_csv(data_path + 'tlacuachitos_vip_customers_data.csv')
df_data.head()

,CustomerID,Age,Income,Tenure,Education,Industry,Geographic Location,Churn_Risk,Cohort
0,1,56,52752.677346,3,Master,Technology,Europe,1,2023-08-31
1,2,69,55297.364348,6,Bachelor,Technology,South America,0,2021-08-31
2,3,46,57978.753383,3,Bachelor,Finance,Europe,1,2019-05-31
3,4,32,60445.266900,3,High School,Education,South America,1,2021-02-28
4,5,60,57741.870929,5,Bachelor,Entertainment,Asia,0,2018-10-31


In [3]:
df_transactions = pd.read_csv(data_path + 'tlacuachitos_vip_transactions.csv')
df_transactions.head()

,CustomerID,TransactionDate,TransactionAmount
0,1,2023-10-31,518.444092
1,1,2024-07-31,353.796197
2,1,2024-01-31,38.206591
3,1,2024-06-30,724.929423
4,2,2022-02-28,145.616000


In [4]:
df_transactions['TransactionDate'] = pd.to_datetime(df_transactions['TransactionDate'])

In [5]:
df_data['Cohort'] = pd.to_datetime(df_data['Cohort'])

In [6]:
snapshot_date = df_transactions['TransactionDate'].max()

In [7]:
df_data['customer_tenure'] = (snapshot_date.year - df_data['Cohort'].dt.year) * 12 + (
    snapshot_date.month - df_data['Cohort'].dt.month)
df_data['customer_tenure']

,customer_tenure
0,12
1,36
2,63
3,42
4,70
...,...
1138,56
1139,22
1140,76
1141,70


In [8]:
df_master = df_transactions.merge(df_data, on = 'CustomerID')
df_master.head()

,CustomerID,TransactionDate,TransactionAmount,Age,Income,Tenure,Education,Industry,Geographic Location,Churn_Risk,Cohort,customer_tenure
0,1,2023-10-31,518.444092,56,52752.677346,3,Master,Technology,Europe,1,2023-08-31,12
1,1,2024-07-31,353.796197,56,52752.677346,3,Master,Technology,Europe,1,2023-08-31,12
2,1,2024-01-31,38.206591,56,52752.677346,3,Master,Technology,Europe,1,2023-08-31,12
3,1,2024-06-30,724.929423,56,52752.677346,3,Master,Technology,Europe,1,2023-08-31,12
4,2,2022-02-28,145.616000,69,55297.364348,6,Bachelor,Technology,South America,0,2021-08-31,36


In [9]:
df_master['customer_tenure_on_transaction'] = (df_master['TransactionDate'].dt.year - df_master['Cohort'].dt.year)*12 + (
    df_master['TransactionDate'].dt.month - df_master['Cohort'].dt.month)
df_master.head()

,CustomerID,TransactionDate,TransactionAmount,Age,Income,Tenure,Education,Industry,Geographic Location,Churn_Risk,Cohort,customer_tenure,customer_tenure_on_transaction
0,1,2023-10-31,518.444092,56,52752.677346,3,Master,Technology,Europe,1,2023-08-31,12,2
1,1,2024-07-31,353.796197,56,52752.677346,3,Master,Technology,Europe,1,2023-08-31,12,11
2,1,2024-01-31,38.206591,56,52752.677346,3,Master,Technology,Europe,1,2023-08-31,12,5
3,1,2024-06-30,724.929423,56,52752.677346,3,Master,Technology,Europe,1,2023-08-31,12,10
4,2,2022-02-28,145.616000,69,55297.364348,6,Bachelor,Technology,South America,0,2021-08-31,36,6


In [10]:
cltv_24_months = df_master.query(
    "customer_tenure > 24 and customer_tenure_on_transaction <= 24").groupby(
    'CustomerID')['TransactionAmount'].sum().reset_index()
cltv_24_months

,CustomerID,TransactionAmount
0,2,145.616000
1,3,487.285458
2,4,1095.025887
3,5,1685.260152
4,6,728.364381
...,...,...
857,1136,189.961932
858,1138,2375.255820
859,1139,1327.509070
860,1141,718.548745


In [11]:
categorical_features = ['Education','Industry','Geographic Location']
numerical_features = ['Age','Income','Tenure']

data_encoded = pd.get_dummies(df_data[['CustomerID'] + categorical_features], columns=categorical_features, drop_first=True)


df_data_with_encoded = df_data[['CustomerID'] + numerical_features].merge(
    data_encoded, on = 'CustomerID'
)

df_to_model = cltv_24_months.merge(df_data_with_encoded, on = 'CustomerID')
df_to_model

,CustomerID,TransactionAmount,Age,Income,Tenure,Education_High School,Education_Master,Education_PhD,Industry_Entertainment,Industry_Finance,Industry_Healthcare,Industry_Technology,Geographic Location_Australia,Geographic Location_Europe,Geographic Location_North America,Geographic Location_South America
0,2,145.616000,69,55297.364348,6,False,False,False,False,False,False,True,False,False,False,True
1,3,487.285458,46,57978.753383,3,False,False,False,False,True,False,False,False,True,False,False
2,4,1095.025887,32,60445.266900,3,True,False,False,False,False,False,False,False,False,False,True
3,5,1685.260152,60,57741.870929,5,False,False,False,True,False,False,False,False,False,False,False
4,6,728.364381,25,57132.404622,3,False,True,False,False,False,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,1136,189.961932,67,59279.959062,4,True,False,False,False,True,False,False,False,False,False,False
858,1138,2375.255820,28,59996.754751,5,False,False,False,True,False,False,False,False,False,True,False
859,1139,1327.509070,26,45236.402018,8,False,False,False,False,False,False,False,False,True,False,False
860,1141,718.548745,29,57043.387540,5,True,False,False,False,True,False,False,False,False,True,False


In [12]:
columns_from_education = df_to_model.loc[:, 'Education_High School':].select_dtypes(include='bool').columns
df_to_model[columns_from_education] = df_to_model[columns_from_education].astype(int)
df_to_model

,CustomerID,TransactionAmount,Age,Income,Tenure,Education_High School,Education_Master,Education_PhD,Industry_Entertainment,Industry_Finance,Industry_Healthcare,Industry_Technology,Geographic Location_Australia,Geographic Location_Europe,Geographic Location_North America,Geographic Location_South America
0,2,145.616000,69,55297.364348,6,0,0,0,0,0,0,1,0,0,0,1
1,3,487.285458,46,57978.753383,3,0,0,0,0,1,0,0,0,1,0,0
2,4,1095.025887,32,60445.266900,3,1,0,0,0,0,0,0,0,0,0,1
3,5,1685.260152,60,57741.870929,5,0,0,0,1,0,0,0,0,0,0,0
4,6,728.364381,25,57132.404622,3,0,1,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,1136,189.961932,67,59279.959062,4,1,0,0,0,1,0,0,0,0,0,0
858,1138,2375.255820,28,59996.754751,5,0,0,0,1,0,0,0,0,0,1,0
859,1139,1327.509070,26,45236.402018,8,0,0,0,0,0,0,0,0,1,0,0
860,1141,718.548745,29,57043.387540,5,1,0,0,0,1,0,0,0,0,1,0


In [13]:
# Estandarizar

#columns_to_scale = ['Age', 'Income', 'Tenure']
#scaler = StandardScaler()
#df_to_model[columns_to_scale] = scaler.fit_transform(df_to_model[columns_to_scale])
#df_to_model

In [15]:
X1 = df_to_model[['Age', 'Income','Tenure','Education_High School','Education_Master','Education_PhD','Industry_Entertainment','Industry_Finance',
                 'Industry_Healthcare','Industry_Technology','Geographic Location_Australia','Geographic Location_Europe',
                 'Geographic Location_North America','Geographic Location_South America']]
y1 = df_to_model['TransactionAmount']

In [16]:
# Agregar la constante (intercepto) a las variables independientes
X1 = sm.add_constant(X1)

# Ajustar el modelo usando OLS
model_1 = sm.OLS(y1, X1)
results = model_1.fit()

# Mostrar el resumen del modelo
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:      TransactionAmount   R-squared:                       0.129
Model:                            OLS   Adj. R-squared:                  0.115
Method:                 Least Squares   F-statistic:                     8.979
Date:                Tue, 08 Oct 2024   Prob (F-statistic):           1.53e-18
Time:                        18:22:08   Log-Likelihood:                -6767.4
No. Observations:                 862   AIC:                         1.356e+04
Df Residuals:                     847   BIC:                         1.364e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const 